In [2]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

import warnings
warnings.filterwarnings(action='ignore')

# 웹수집 방법

- 웹스크랩핑 : requests, urllib, bs4, selenium (html, css 지식 필요)
- 공공데이타 포털 제공 => csv, excel
- API 서비스 => 네이버, 카카오, 통계청, 공공데이타포털 (xml, json 지식 필요)
- 판다스 금융라이브러리 (주식, 비트코인, 환율, 원자재....)

# 웹데이타 수집 과정

- 웹에 필요한 URL 요청 (관련 모듈 => requests, urllib)
- response 객체 => html 페이지
- 파싱(bs4, selenium)
- 리스트, 딕셔너리, 판다스의 데이타프레임 형태로 저장
- csv 파일로 저장하거나 데이타베이스에 저장

### requests 모듈을 이용하여 특정 url 주소의 페이지 저장하기 

- !pip install requests

In [4]:
# pip list

In [1]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

In [8]:
# url 요청 => response 객체 생성 
url = 'http://google.com'
res = requests.get(url)

# 요청 확인 
print(res.headers)

# response 객체 => 텍스트화 
print(f'res = {res}')
# res = <Response [200]>

html_str = res.text
print('='*50)
html_str

{'Date': 'Thu, 18 Nov 2021 06:00:36 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=ISO-8859-1', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'Content-Length': '6500', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2021-11-18-06; expires=Sat, 18-Dec-2021 06:00:36 GMT; path=/; domain=.google.com; Secure, NID=511=uQovDvpBpj8YPVSD6kytnwdyi5gvX2fouyQKM4tl9LxDDKVyouXxZp8nXWyzWEYdHBCebvA8jVBkymNWi0uvc8ete187hQ3f0lRIYn0W9Gi3KnLNA_6skArkwhWNfi2iMhyfK9EvsXNYcPCo-MvjR1oOe2VbQCWUNq8on2DZPDk; expires=Fri, 20-May-2022 06:00:36 GMT; path=/; domain=.google.com; HttpOnly'}
res = <Response [200]>


'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="I/eChlDKhZm4n49yrikmYQ==">(function(){window.google={kEI:\'BOyVYaPdJMy3mAWQuJ2ACw\',kEXPI:\'0,202372,1100104,60,56873,6059,206,2415,2389,2316,383,246,5,1354,4012,1238,1122516,1197702,699,328866,51224,16114,28684,17572,4858,1362,9291,3025,17583,4020,978,13228,3847,4192,6430,14763,7059,919,2855,3819,1279,2212,241,289,149,1103,840,6297,109,4011,2023,2297,6344,8326,3227,2845,7,12354,5096,16320,908,2,3555,13142,3,576,6459,149,12289,1686,4,1253,275,2304,6462,577,4683,2015,18375,2658,6701,655,31,13628,1592,714,2131,13406,3380,5824,2533,4094,3138,6,908,3,3541,1,11943,4581,283,912,5992,15331,2,3110,2,3036,10986,1931,3911,1678,743,5853,9320,134,1009,1160,1266,5434,2379,2720,8595,684,4890,4127,2,6,7719,4568,2577,3128

In [9]:
# 폴더 생성 후 html 페이지 저장 
try:
    os.mkdir('output2')
except:
    pass

with open('output2/google.html', 'w') as f:
    f.write(html_str)

In [10]:
ls output2

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2819-FF50

 C:\workspace211115\output2 디렉터리

2021-11-18  오후 03:02    <DIR>          .
2021-11-18  오후 03:02    <DIR>          ..
2021-11-18  오후 03:02            15,251 google.html
               1개 파일              15,251 바이트
               2개 디렉터리  73,217,404,928 바이트 남음


## bs4를 이용해서 파싱 처리하기 

## 파싱(Parsing)
가공되지 않은 데이타에서 원하는 정보를 추출하는 과정

## BeautifulSoup
- 외부 라이브러리
- pip list 명령으로 설치 확인 : beautifulsoup4
- 없다면 설치 : pip install beautifulsoup4
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.html


# 뷰티풀숩 임포트
```
from bs4 import BeautifulSoup
```

### BeautifulSoup을 이용한 작업 과정
- requests를 이용해서 url 요청 => response 객체 => html화 (텍스트)
- html화된 자료 => 뷰티풀숩 객체화
- 뷰티풀숩의 다양한 함수를 이용해서 자료변환(리스트나 딕셔너리 데이타프레임등)
- csv 파일등로 저장 (판다스를 이용)

###  샘플 스크래핑 url 지정 
url = 'http://pythonscraping.com/pages/warandpeace.html'

In [4]:
# url 요청 => response 객체 생성 
url = 'http://pythonscraping.com/pages/warandpeace.html'
res = requests.get(url)

# response 객체 => 텍스트화 
html_str = res.text
print('='*50)
# html_str

### html화된 자료 => 뷰티풀숩 객체화

- BeautifulSoup(html변수 , 해석기 )
- 해석기 : html.parser/html5/xml/lxml
- soup = BeautifulSoup(html_str, 'html.parser' )

In [6]:
soup = BeautifulSoup(html_str, 'html.parser')
# soup

### bs4의 메서드 이용하기 

##### find_all(태그) => 리스트 형태
- soup.find_all('span')

#### find(태그)
- 태그가 여러개일 경우에는 첫번째 태그만 반환 
- soup.find('p')

#### find(id=아이디명)
soup.find(id='text').text

#### find(class_=클래스명)
soup.find(class_='green')

#### 특정 태그의 속성값 출력
get(속성명)



In [10]:
# 특정태그 한개 정보 추출 
print(soup.find('h1'))

# 삽입된 텍스트만 => .text
print(soup.find('h1').text)

<h1>War and Peace</h1>
War and Peace


In [12]:
# 모든 span 태그 => 리스트 
span_list = soup.find_all('span')
len(span_list)

75

In [14]:
span_list[5]

<span class="green">St. Petersburg</span>

In [16]:
# 텍스트만 리스트로 저장 
span_text_list = []
for item in span_list:
    # print(item.text)
    span_text_list.append(item.text)

In [18]:
span_text_list[:3]

["Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy 'faithful slave,' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.",
 'Anna\nPavlovna Scherer',
 'Empress Marya\nFedorovna']

In [19]:
# find_all(class_=클래스명)
# green 클래스가 적용된 태그 => 리스트
green_list = soup.find_all(class_='green')
len(green_list)

41

In [20]:
green_list[0]

<span class="green">Anna
Pavlovna Scherer</span>

### select() 와 select_one()

- select(선택자) => 리스트
- select_one(선택자) => 1개의 요소

In [22]:
# red 클래스가 적용된 태그 => 리스트
red_list = soup.select('.red')
len(red_list)

34

In [23]:
red_one = soup.select_one('.red')
red_one

<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>

#### 속성값 추출 
 - get(속성명)
 - ['속성명']

In [26]:
red_one.text

"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy 'faithful slave,' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news."

In [27]:
red_one.get('class')

['red']

In [30]:
red_one['class']

['red']

In [28]:
red_one

<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>

In [7]:
soup

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

# 예제 1

알렉사 사이트의 50위  https://www.alexa.com/topsites


In [32]:
# url 요청 => response 객체 생성 
url = 'https://www.alexa.com/topsites/countries/KR'
res = requests.get(url)

# response 객체 => 텍스트화 
html_str = res.text
# html_str

In [35]:
# soup 객체화 
soup = BeautifulSoup(html_str, 'html.parser')
# soup

In [36]:
# 리스트로 저장 
div_list = soup.find_all(class_='tr site-listing')
len(div_list)

50

In [37]:
div_list[-1]

<div class="tr site-listing">
<div class="td">50</div>
<div class="td DescriptionCell">
<p class="">
<a href="/siteinfo/ebay.com">Ebay.com</a>
</p>
</div>
<div class="td right"><p>9:60</p></div>
<div class="td right"><p>7.80</p></div>
<div class="td right"><p>16.70%</p></div>
<div class="td right"><p>81,753</p></div>
</div>

In [44]:
# 순위 rank
print(div_list[-1].select_one('.td').text)
# 사이트 제목 title
print(div_list[-1].find('a').text)
# 사이트 URL 
print(div_list[-1].find('a')['href'])
# daily_time
print(div_list[-1].find_all(class_='td right')[0].text)
# daily_pageviews
print(div_list[-1].find_all(class_='td right')[1].text)

50
Ebay.com
/siteinfo/ebay.com
9:60
7.80


In [45]:
# 2차원 리스트 저장 
temp_list = []
for row in div_list:
    rank = row.select_one('.td').text
    title = row.find('a').text
    daily_time = row.find_all(class_='td right')[0].text
    daily_pageviews = row.find_all(class_='td right')[1].text
    temp_list.append([rank, title, daily_time, daily_pageviews])

In [46]:
# 데이타프레임으로 저장 
df_alexa = pd.DataFrame(temp_list, columns=['rank', 'title', 'daily_time', 'daily_pageviews'])
df_alexa.head()

,rank,title,daily_time,daily_pageviews
0,1,Google.com,17:29,18.51
1,2,Naver.com,15:13,13.10
2,3,Youtube.com,19:33,10.57
3,4,Daum.net,12:09,8.66
4,5,Tistory.com,5:56,4.03


In [47]:
df_alexa.tail()

,rank,title,daily_time,daily_pageviews
45,46,Jobkorea.co.kr,5:51,5.70
46,47,Hankyung.com,2:16,2.54
47,48,Yna.co.kr,4:21,3.77
48,49,Inven.co.kr,6:18,6.41
49,50,Ebay.com,9:60,7.80


In [49]:
# csv 파일로 저장 
df_alexa.to_csv('output2/alexa.csv', index=False)

In [50]:
ls output2

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2819-FF50

 C:\workspace211115\output2 디렉터리

2021-11-18  오후 04:33    <DIR>          .
2021-11-18  오후 04:33    <DIR>          ..
2021-11-18  오후 04:33             1,308 alexa.csv
2021-11-18  오후 03:02            15,251 google.html
               2개 파일              16,559 바이트
               2개 디렉터리  75,282,919,424 바이트 남음
